# Hands-on argo workflows

## Get your webUI credentials

You can use the output of the cell below to access the webUI at: 
`http://argoworkflows.131.154.96.42.myip.cloud.infn.it/<YOUR JUPYTER USERNAME HERE>`

In [ ]:
import os
with open('/var/run/secrets/kubernetes.io/serviceaccount/token') as token_file:
    token = token_file.read()
argo_token = f"{token}"
user = os.environ.get("JUPYTERHUB_USER")
print(f"Jupyter Username {user}")
print(f"Bearer {token}")

## Create your first workflow

Let's create a yaml file `whalesay.yaml` with our first Argo worklow. It will simply make a whale speaking :)

```yaml
 
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: hello-world-parameters-
spec:
  entrypoint: whalesay
  arguments:
    parameters:
    - name: message
      value: Message string default value 
  templates:
  - name: whalesay
    inputs:
      parameters:
      - name: message
    container:
      image: docker/whalesay
      command: [cowsay]
      args: ["{{inputs.parameters.message}}"] 
```

### Submit and watch your first workflow from a terminal

```bash
argo submit whalesay.yaml --watch
```

### Let's now explore what happened

--> You can now look at the webUI to see all the details and logs of the workflow
--> Also, you will be able to find the logs on your S3 storage at https://console.131.154.96.42.myip.cloud.infn.it/

### Create a chain of tasks

Now we are going to create a more complex workflow, where we put more tasks in a chain. 

The example consists in making a first task tossing a coin and subsequent ones acting differentely depending on the outcome.

### Flip coin example

Let's create a yaml file flipcoin.yaml with our Argo worklow.

```yaml
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: conditional-parameter-
  labels:
    workflows.argoproj.io/test: "true"
  annotations:
    workflows.argoproj.io/version: '>= 3.1.0'
spec:
  entrypoint: main
  templates:
    - name: main
      steps:
        - - name: flip-coin
            template: flip-coin
        - - name: heads
            template: heads
            when: "{{steps.flip-coin.outputs.result}} == heads"
          - name: tails
            template: tails
            when: "{{steps.flip-coin.outputs.result}} == tails"
      outputs:
        parameters:
          - name: stepresult
            valueFrom:
              expression: "steps['flip-coin'].outputs.result == 'heads' ? steps.heads.outputs.result : steps.tails.outputs.result"
    - name: flip-coin
      script:
        image: python:alpine3.6
        command: [ python ]
        source: |
          import random
          print("heads" if random.randint(0,1) == 0 else "tails")
    - name: heads
      script:
        image: python:alpine3.6
        command: [ python ]
        source: |
          print("heads")
    - name: tails
      script:
        image: python:alpine3.6
        command: [ python ]
        source: |
          print("tails")
```

and then again

```bash
argo submit flipcoin.yaml --watch
```

## Extra: define workflows programmatically

Let's now reproduce the flip coin example via the python bindings provided by Hera framework.


In [ ]:
import uuid
from hera import Task
from hera import Workflow, WorkflowService


def random_code():
    import random

    res = "heads" if random.randint(0, 1) == 0 else "tails"
    print(res)


def heads():
    print("it was heads")


def tails():
    print("it was tails")

svc = WorkflowService(
    token=argo_token,
    host="http://argoworkflows.131.154.96.42.myip.cloud.infn.it/<YOUR JUPYTER USERNAME HERE>",
    namespace=f"user-{user}"
)

wf_name = f"coin-flip-{uuid.uuid4()}"

# assumes you used `hera.set_global_token` and `hera.set_global_host` so that the workflow can be submitted
with Workflow(wf_name, service=svc) as w:
    r = Task("r", random_code)
    h = Task("h", heads)
    t = Task("t", tails)

    h.on_other_result(r, "heads")
    t.on_other_result(r, "tails")

wf = w.create()

### Monitor the workflow status

You can either go on the WebUI or programmatically via:

In [ ]:
import time

while svc.get_workflow_status(wf_name) not in ["Succeeded", "Failed", "Error"]:
    print(svc.get_workflow_status(wf_name))
    time.sleep(5)

print("Worflow ended in status:", svc.get_workflow_status(wf_name))

### Use a custom image

You can use a custom image in a task with something like:

```python
from hera import ImagePullPolicy, Task, Workflow

with Workflow("pipeline-image-testing", service=svc) as w:
    # This can be used when you have your own custom image
    # Image_pull_policy is set to Never because on localhost when you test
    # you don't need to pull the image
    Task(
        "workflow-with-custom-image",
        image="my-custom-image-name:latest",
        image_pull_policy=ImagePullPolicy.Never,
        command=["python", "-m", "src.pipeline_example"],
    )

w.create()
```

### Artifacts

Let's investigate how we can make two task communicate with data written on a file.

Notice that the intermediate file will be stored as artifact also on the S3 storage.

In [ ]:
from hera import Artifact, Task, Workflow


def writer():
    with open("/file", "w+") as f:
        f.write("Hello, world!")


def consumer():
    with open("/file", "r") as f:
        print(f.readlines())  # prints `Hello, world!` to `stdout`


# assumes you used `hera.set_global_token` and `hera.set_global_host` so that the workflow can be submitted
with Workflow(f"artifact-{uuid.uuid4()}", service=svc) as w:
    w_t = Task("writer", writer, outputs=[Artifact("test", "/file")])
    c_t = Task(
        "consumer",
        consumer,
        inputs=[w_t.get_artifact("test")],
    )

    w_t >> c_t

w.create()

### Workflow templates (BONUS)

If you want to reuse the structure of a worflow more than once, you can store a workflow template.

For instance, let's create a flipcoin template


In [ ]:
from hera import Task, WorkflowTemplate


def random_code():
    import random

    res = "heads" if random.randint(0, 1) == 0 else "tails"
    print(res)


def heads():
    print("it was heads")


def tails():
    print("it was tails")


with WorkflowTemplate(f"hera-workflow-templates", dag_name="coin-flip", service=svc) as w:
    r = Task("r", random_code)
    Task("h", heads).on_other_result(r, "heads")
    Task("t", tails).on_other_result(r, "tails")

w.create()

### Reuse a workflow template

Once you created a template you can then reuse it with some like:

In [ ]:
from hera import Task, TemplateRef, Workflow

# The name of the DAG template is either the name of WorkflowTemplate (default), or the `dag_name`
with Workflow(f"workflow-with-template-ref-{uuid.uuid4()}", service=svc) as w:
    Task("coin-flip", template_ref=TemplateRef(name="hera-workflow-templates", template="coin-flip"))

w.create()

## Homework (TBD)


